In [1]:
# Import libraries
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout

In [3]:
df_for_training = pd.read_csv("C:/Users/Clyden Pacheco/Documents/Excel/df_for_training.csv")

In [4]:
df_for_training.shape

(24548, 57)

In [9]:
df_for_training

,Open,High,Low,Close,Adj Close,Volume,Sentiments,Stock_ADANIENT.NS,Stock_ADANIPORTS.NS,Stock_APOLLOHOSP.NS,...,Stock_SHRIRAMFIN.NS,Stock_SUNPHARMA.NS,Stock_TATACONSUM.NS,Stock_TATAMOTORS.NS,Stock_TATASTEEL.NS,Stock_TCS.NS,Stock_TECHM.NS,Stock_TITAN.NS,Stock_ULTRACEMCO.NS,Stock_WIPRO.NS
0,0.344884,0.313330,0.378343,0.339727,0.339412,0.024234,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.339109,0.298554,0.344151,0.298220,0.297923,0.034739,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.372607,0.333843,0.339674,0.337489,0.337175,0.143612,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.354257,0.314964,0.379393,0.333502,0.333189,0.024412,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.349835,0.314130,0.347202,0.299314,0.299016,0.043164,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24543,0.490687,0.485012,0.493933,0.470557,0.472697,0.044699,0.256683,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24544,0.467802,0.459897,0.358522,0.438704,0.440973,0.115144,0.166468,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24545,0.434806,0.509047,0.480143,0.510171,0.512151,0.071241,0.367750,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24546,0.515966,0.546854,0.554881,0.561028,0.562802,0.096620,0.533555,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
df_for_training.columns[:7]

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Sentiments'], dtype='object')

In [7]:
# Function to create sequences
def singleStepSampler(df, window):
	xRes = []
	yRes = []
	for i in range(0, len(df) - window):
		res = []
		for j in range(0, window):
			r = []
			for col in df.columns:
				r.append(df[col][i + j])
			res.append(r)
		xRes.append(res)
		yRes.append(df[['Open', 'Close', 'High', 'Low', 'Adj Close', 'Volume', 'Sentiments']].iloc[i + window].values)
	return np.array(xRes), np.array(yRes)

In [8]:
# Dataset splitting
SPLIT = 1.0
(xVal, yVal) = singleStepSampler(df_for_training, 20)
X_train = xVal[:int(SPLIT * len(xVal))]
y_train = yVal[:int(SPLIT * len(yVal))]
X_test = xVal[int(SPLIT * len(xVal)):]
y_test = yVal[int(SPLIT * len(yVal)):]
print(xVal.shape)
print(yVal.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24528, 20, 57)
(24528, 7)
(24528, 20, 57)
(24528, 7)
(0, 20, 57)
(0, 7)


In [15]:
# Building the model

multivariate_lstm = keras.Sequential()

# First LSTM layer to capture short-term dependencies within the window
multivariate_lstm.add(keras.layers.LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
multivariate_lstm.add(keras.layers.Dropout(0.2))  # Dropout for regularization

# Second LSTM layer to capture potential hierarchical features
multivariate_lstm.add(keras.layers.LSTM(100))  # Experiment with different units here
multivariate_lstm.add(keras.layers.Dropout(0.2))  # Dropout for regularization

# Dense layer with ReLU activation for non-linearity
multivariate_lstm.add(keras.layers.Dense(7, activation='relu'))

# Compile the model with Adam optimizer and Mean Squared Error loss
multivariate_lstm.compile(loss = 'MeanSquaredError', metrics=['MAE'], optimizer='Adam')

# Print model summary
multivariate_lstm.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 20, 200)           206400    
                                                                 
 dropout_6 (Dropout)         (None, 20, 200)           0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               120400    
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 7)                 707       
                                                                 
Total params: 327507 (1.25 MB)
Trainable params: 327507 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Fitting the data to the model
history = multivariate_lstm.fit(X_train, y_train, epochs=100)

Epoch 1/100


767/767 [==============================] - 19s 21ms/step - loss: 0.0087 - MAE: 0.0601
Epoch 2/100
767/767 [==============================] - 16s 21ms/step - loss: 0.0050 - MAE: 0.0438
Epoch 3/100
767/767 [==============================] - 16s 20ms/step - loss: 0.0043 - MAE: 0.0390
Epoch 4/100
767/767 [==============================] - 16s 20ms/step - loss: 0.0040 - MAE: 0.0365
Epoch 5/100
767/767 [==============================] - 16s 20ms/step - loss: 0.0038 - MAE: 0.0348
Epoch 6/100
767/767 [==============================] - 16s 21ms/step - loss: 0.0037 - MAE: 0.0336
Epoch 7/100
767/767 [==============================] - 16s 21ms/step - loss: 0.0035 - MAE: 0.0328
Epoch 8/100
767/767 [==============================] - 16s 21ms/step - loss: 0.0035 - MAE: 0.0325
Epoch 9/100
767/767 [==============================] - 16s 20ms/step - loss: 0.0035 - MAE: 0.0322
Epoch 10/100
767/767 [==============================] - 16s 21ms/step - loss: 0.0034 - MAE: 0.0321
Epoch 11/100
767/

In [17]:
# Save the model to a specific path
model_path = 'C:/Users/Clyden Pacheco/Documents/Models/multivariate_lstm_model3.h5'
multivariate_lstm.save(model_path)

C:\Users\Clyden Pacheco\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
